In [ ]:
!pip install deepgram-sdk python-dotenv

In [ ]:
import os
import threading
from dotenv import load_dotenv
from deepgram.core.events import EventType
from deepgram import DeepgramClient
from deepgram.speak.v1.types.speak_v1text import SpeakV1Text

load_dotenv()
API_KEY = os.getenv("DEEPGRAM_API_KEY")
OUTPUT_FILE = "tts_output.wav"

In [ ]:
def stream_tts_to_file():
    try:
        deepgram = DeepgramClient(api_key=API_KEY)
        
        with deepgram.speak.v1.connect(
            model="aura-asteria-en",
            encoding="linear16",
            sample_rate=24000,
        ) as dg_connection:

            audio_file = open(OUTPUT_FILE, "wb")
            flush_complete = threading.Event()
            
            def on_open(self, *args, **kwargs):
                print(f"\n🔌 WebSocket Connection Opened")

            def on_message(self, message, *args, **kwargs):
                if isinstance(message, bytes):
                    audio_file.write(message)
                    print(f"🎵 Received Audio Bytes: {len(message)}")
                else:
                    print(f"📩 Received Event: {message}")

            def on_close(self, *args, **kwargs):
                print("🔌 WebSocket Connection Closed")
                audio_file.close()

            # Corrected signature to catch whatever positional or keyword arguments the SDK throws
            def on_error(self, *args, **kwargs):
                print(f"❌ WebSocket Error: args={args}, kwargs={kwargs}")
                
            dg_connection.on(EventType.OPEN, on_open)
            dg_connection.on(EventType.MESSAGE, on_message)
            dg_connection.on(EventType.CLOSE, on_close)
            dg_connection.on(EventType.ERROR, on_error)
            
            try:
                dg_connection.start()
            except AttributeError:
                pass 

            print("\nSpeaking: Hello! This audio is dynamically beamed from the cloud as it is generated.")
            
            dg_connection.send_text(SpeakV1Text(text="Hello! This audio is dynamically beamed from the cloud as it is generated."))
            dg_connection.send_flush()
            
            dg_connection.start_listening()
            print(f"\nSaved Raw Output to {OUTPUT_FILE} (24000Hz 16-bit PCM Audio).")
            
    except Exception as e:
        print(f"Exception: {e}")

stream_tts_to_file()